In [1]:
# import dependencies
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from fbprophet import Prophet

from fbprophet.plot import plot_plotly
import plotly.offline as py

import plotly.offline as py
import plotly.graph_objs as go

In [2]:
# Create your connections with indoor and outdoor databases
Ocnx = sqlite3.connect('Outdoor.db')
Icnx = sqlite3.connect('Indoor.db')
Outdoor_df = pd.read_sql_query("SELECT * FROM BME_DATA", Ocnx)
Outdoor_df = Outdoor_df.iloc[1:]
Indoor_df = pd.read_sql_query("SELECT * FROM BME_DATA", Icnx)
Indoor_df = Indoor_df.iloc[1:]

In [3]:
# Cleaning up Outdoor Data
Outdoor_df['TIME_STAMP'] = pd.to_datetime(Outdoor_df['TIME_STAMP'])
Outdoor_df['TIME_STAMP'] = Outdoor_df['TIME_STAMP'].dt.round('60min')
Outdoor_df = Outdoor_df.groupby(['TIME_STAMP'], as_index=False)["TEMPERATURE","GAS","HUMIDITY", "PRESSURE", "ALTITUDE"].mean()
Outdoor_df = Outdoor_df.loc[:,["TIME_STAMP", "TEMPERATURE", "GAS", "HUMIDITY", "PRESSURE"]]
Outdoor_df["GAS"] = round(Outdoor_df["GAS"],1)
Outdoor_df["HUMIDITY"] = round(Outdoor_df["HUMIDITY"],1)
Outdoor_df["TEMPERATURE"] = round(Outdoor_df["TEMPERATURE"],1)
Outdoor_df["PRESSURE"] = round(Outdoor_df["PRESSURE"],1)
Outdoor_df['TEMPERATURE'] = round((Outdoor_df['TEMPERATURE']* 9/5) + 32)
Outdoor_df.head()

,TIME_STAMP,TEMPERATURE,GAS,HUMIDITY,PRESSURE
0,2019-05-12 19:00:00,75.0,367019.2,84.8,975.0
1,2019-05-12 20:00:00,73.0,195655.8,89.4,975.2
2,2019-05-12 21:00:00,72.0,182528.4,92.0,975.7
3,2019-05-12 22:00:00,72.0,342664.5,91.5,976.1
4,2019-05-12 23:00:00,71.0,268596.2,90.1,976.8


In [4]:
# Cleaning up Indoor Data
Indoor_df['TIME_STAMP'] = pd.to_datetime(Indoor_df['TIME_STAMP'])
Indoor_df['TIME_STAMP'] = Indoor_df['TIME_STAMP'].dt.round('60min')
Indoor_df = Indoor_df.groupby(['TIME_STAMP'], as_index=False)["TEMPERATURE","GAS","HUMIDITY", "PRESSURE", "ALTITUDE"].mean()
Indoor_df = Indoor_df.loc[:,["TIME_STAMP", "TEMPERATURE", "GAS", "HUMIDITY", "PRESSURE"]]
Indoor_df["GAS"] = round(Indoor_df["GAS"],1)
Indoor_df["HUMIDITY"] = round(Outdoor_df["HUMIDITY"],1)
Indoor_df["TEMPERATURE"] = round(Indoor_df["TEMPERATURE"],1)
Indoor_df["PRESSURE"] = round(Indoor_df["PRESSURE"],1)
Indoor_df['TEMPERATURE'] = round((Indoor_df['TEMPERATURE']* 9/5) + 32)
Indoor_df.head()

,TIME_STAMP,TEMPERATURE,GAS,HUMIDITY,PRESSURE
0,2019-05-12 19:00:00,75.0,4203911.7,84.8,972.9
1,2019-05-12 20:00:00,75.0,1188237.0,89.4,973.0
2,2019-05-12 21:00:00,74.0,723933.0,92.0,973.4
3,2019-05-12 22:00:00,75.0,723933.0,91.5,973.9
4,2019-05-12 23:00:00,75.0,723933.0,90.1,974.6


In [5]:
# Merging Indor and Outdoor (on time stamp)
master_df = pd.merge(Outdoor_df, Indoor_df, on = "TIME_STAMP", how = "left", suffixes=("_Out","_In"))
master_df= master_df.dropna()
master_df.head()
master_df.to_csv("Merged_Data.csv", sep=',', encoding='utf-8')

In [6]:
# Creating Json for chart1
prep_json = master_df.loc[:,["TIME_STAMP", "TEMPERATURE_Out","TEMPERATURE_In"]]
json_df = prep_json.to_json(orient='records')

In [ ]:
# Creating indoor and Outdoor Datasets
outdoor = master_df.loc[:,["TIME_STAMP", "TEMPERATURE_Out"]]
indoor = master_df.loc[:,["TIME_STAMP", "TEMPERATURE_In"]]
outdoor.columns = ['ds', 'y']
indoor.columns = ['ds', 'y']

In [ ]:
# Export to csv
outdoor.to_csv("indoor_data.csv", sep=',', encoding='utf-8')
indoor.to_csv("outdoor_data.csv", sep=',', encoding='utf-8')

In [ ]:
# Building the model for inodor and outdoor
od = Prophet()
od.fit(outdoor)
ind = Prophet()
ind.fit(indoor)

In [ ]:
# Outdoor forecasting
od_future = od.make_future_dataframe(periods= 1)
od_forecast = od.predict(od_future)
# Export to csv
od_forecast.to_csv("od_forecast.csv", sep=',', encoding='utf-8')
od_forecast.head()

In [ ]:
# Indoor forecasting
ind_future = ind.make_future_dataframe(periods= 1)
ind_forecast = ind.predict(ind_future)
ind_forecast.to_csv("ind_forecast.csv", sep=',', encoding='utf-8')
ind_forecast.head()

In [ ]:
# Outdoor plot with forecasting
od_plot = od.plot(od_forecast)

In [ ]:
# Indoor plot with forecasting
ind_plot = ind.plot(ind_forecast)

In [ ]:
# Print outdoor trend and daily component
od_fig1 = od.plot_components(od_forecast)

In [ ]:
# Print indoor trend and daily component
ind_fig1 = ind.plot_components(ind_forecast)

In [ ]:
# Print outdoor 
py.init_notebook_mode()
od_fig = plot_plotly(od, od_forecast)  # This returns a plotly Figure
py.iplot(od_fig)

In [ ]:
# Print indoor plotly figure
ind_fig = plot_plotly(ind, ind_forecast)  # This returns a plotly Figure
py.iplot(ind_fig)

In [ ]:
# Producing full plot for outdoor data
py.iplot([
    go.Scatter(x=outdoor['ds'], y=outdoor['y'], name='y'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['yhat'], name='yhat'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=od_forecast['ds'], y=od_forecast['trend'], name='Trend')
])

In [ ]:
# Producing full plot for indoor data
py.iplot([
    go.Scatter(x=indoor['ds'], y=indoor['y'], name='y'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['yhat'], name='yhat'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=ind_forecast['ds'], y=ind_forecast['trend'], name='Trend')
])